# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

df = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [2]:
#investing dataset
df.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [3]:
#more investigation
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

# Q1 What test is appropriate for this problem? Does CLT apply?

CLT applies because our sample size is large and we can assume that we are comparing two independent samples (i.e. white-sounding names and black-sounding names). As a result, a two-sample t-test seems appropriate for this problem.

# Q2 - What are the null and alternate hypotheses?

Before we conduct any statistical testing, we'll state the following null and alternate hypotheses:
* Null Hypothesis: There is no significant difference in callback rates between white-sounding and black-sounding names
* Alternate Hypothesis: There is a significant difference in callback rates between white-sounding and black-sounding names

As usual, we will assume a 95% confidence level and reject the null if the test statistic > 1.96 standard deviations with a large p-value

In [4]:
#Bootstrapping approach using permutation method

#Creating helper functions
def perm_sample(df1, df2):
    #Adding both samples together and randomly rearranging them
    df3 = np.concatenate((df1, df2))
    perm_df = np.random.permutation(df3)
    #Creating same length samples using length of original 2 samples
    perm_s1 = perm_df[:len(df1)]
    perm_s2 = perm_df[len(df1):]
    
    return perm_s1, perm_s2

def draw_perm_reps(df1, df2, func, size = 1):
    perm_replicates = np.empty(size)
    
    #Creating boostrap permutation replicates after randomly rearranging both samples
    for i in range(size):
        perm_s1, perm_s2 = perm_sample(df1, df2)
        perm_replicates[i] = func(perm_s1, perm_s2)
    
    return perm_replicates

def diff_of_means(df1, df2):
    diff = np.mean(df1) - np.mean(df2)
    return diff

#Separating main dataset into two samples
df_w = df[df["race"]=='w']["call"]
df_b = df[df["race"]=='b']["call"]
#Dropping race column so concatenation works
#df_w = df_w.drop(["race"], axis = 1)
#df_b = df_b.drop(["race"], axis = 1)

#Calculating difference of mean callback:
empirical_diff_means = diff_of_means(df_w, df_b)

#Creating permutation replicates
perm_replicates = draw_perm_reps(df_w, df_b, diff_of_means, size = 10000)

#Computing p-value and confidence interval
p_value = np.sum(perm_replicates >= empirical_diff_means) / len(perm_replicates)
conf_int = np.percentile(perm_replicates, [2.5, 97.5])
print("P-value is:", p_value)
print("95% Confidence Interval is:", conf_int)

('P-value is:', 0)
('95% Confidence Interval is:', array([-0.01478439,  0.01560576]))


Next. we'll try a Two Proportion Z-test

In [5]:
#Conducting 2 proportion z-test
z_value = 1.96
overall_mean = np.mean(df["call"])

#Computing margin of error and 95% confidence interval
w_mean = np.mean(df_w)
w_std_err = np.sqrt((w_mean * (1 - w_mean) / len(df_w)))

b_mean = np.mean(df_b)
b_std_err = np.sqrt((b_mean * (1 - b_mean) / len(df_b)))

std_err_diff = np.sqrt((w_std_err ** 2 + b_std_err ** 2))
margin_of_err = z_value * std_err_diff
diff_of_means = w_mean - b_mean
conf_int = (diff_of_means - margin_of_err, diff_of_means + margin_of_err)

#Computing Z-score
z_score = (w_mean - b_mean) / \
          (overall_mean * (1 - overall_mean) * ((1 / float(len(df_w))) +  (1 / float(len(df_b)))) ) ** (0.5)

print("Callback rate for white-sounding names is: ", w_mean)
print("Callback rate for black-sounding names is: ", b_mean)

print("Margin of Error is:", margin_of_err)
print("95% Confidence Interval is: ", conf_int)
print("Z-score is:", z_score)

('Callback rate for white-sounding names is: ', 0.09650924056768417)
('Callback rate for black-sounding names is: ', 0.0644763857126236)
('Margin of Error is:', 0.015255406348684322)
('95% Confidence Interval is: ', (0.016777448506376254, 0.0472882612037449))
('Z-score is:', 4.108412148525734)


In [6]:
#To check my math, I used statsmodels package
import statsmodels.api as sm

z_score, p_value = sm.stats.proportions_ztest([sum(df_w), sum(df_b)], [len(df_w), len(df_b)])
print(z_score, p_value)

(4.108412152434346, 3.983886837585077e-05)


Since the two proportional z-test z-score is 4.1 and the p-value was extremely low, we can interpret this as the callback proportions being more than 4 standard deviations away from the mean of 0 difference. We should reject the null hypothesis. In other words, we can confidently say that there is statistical difference in callbacks between black candidates and white candidates

# Q4 - Write a story describing the statistical significance in the context or the original problem.

When you first glance at the difference in callback rates for white-sounding and black-sounding names, the difference is huge - almost 3.3% more white-sounding candidates were selected (9.7% for white-sounding names vs. 6.4% for black-sounding names). 

To test if this was statistically significant, we tested the null hypothesis that there was no sigifnicant diference between the amount of callbacks for white-sounding and black-sounding names. Using frequentist statistical methods, we see that the callback rates would only be the same 4.11 standard deviations away. 

# Q5 - Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

Our analysis doesn't necessarily mean that race/name is the most important factor. It probably is one of the most important, but we wouldn't know for sure unless we looked at other factors within the dataset. For example, it could be the case that education (i.e. another column within the original dataset) is more important than race in getting a callback. 

With our analysis, what we can definitively state is that there is a significant difference in callback rates for white-sounding and black-sounding names.